In [1]:
import pandas as pd

data = pd.read_csv("~/code/mkvph0ch/memobrain/raw_data/OASIS2/oasis_longitudinal_demographics.csv")

data[:10]

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1698,0.701,1.034
5,OAS2_0004,OAS2_0004_MR1,Nondemented,1,0,F,R,88,18,3.0,28.0,0.0,1215,0.710,1.444
6,OAS2_0004,OAS2_0004_MR2,Nondemented,2,538,F,R,90,18,3.0,27.0,0.0,1200,0.718,1.462
7,OAS2_0005,OAS2_0005_MR1,Nondemented,1,0,M,R,80,12,4.0,28.0,0.0,1689,0.712,1.039
8,OAS2_0005,OAS2_0005_MR2,Nondemented,2,1010,M,R,83,12,4.0,29.0,0.5,1701,0.711,1.032
9,OAS2_0005,OAS2_0005_MR3,Nondemented,3,1603,M,R,85,12,4.0,30.0,0.0,1699,0.705,1.033


In [36]:
X.isnull().sum()

MRI ID     0
Group      0
Visit      0
M/F        0
Age        0
EDUC       0
SES       19
MMSE       2
eTIV       0
nWBV       0
ASF        0
dtype: int64

In [40]:
def clean_data(data):
    data=data.drop(columns=['MR Delay', 'Hand', 'Subject ID'])
    data['SES'].fillna(value=int(data.SES.median()), inplace=True)
    data['MMSE'].fillna(value=int(data.MMSE.median()), inplace=True)
    return data

In [43]:
X=clean_data(data).drop(columns=['CDR'])
y=clean_data(data)['CDR']
X

,MRI ID,Group,Visit,M/F,Age,EDUC,SES,MMSE,eTIV,nWBV,ASF
0,OAS2_0001_MR1,Nondemented,1,M,87,14,2.0,27.0,1987,0.696,0.883
1,OAS2_0001_MR2,Nondemented,2,M,88,14,2.0,30.0,2004,0.681,0.876
2,OAS2_0002_MR1,Demented,1,M,75,12,2.0,23.0,1678,0.736,1.046
3,OAS2_0002_MR2,Demented,2,M,76,12,2.0,28.0,1738,0.713,1.010
4,OAS2_0002_MR3,Demented,3,M,80,12,2.0,22.0,1698,0.701,1.034
...,...,...,...,...,...,...,...,...,...,...,...
368,OAS2_0185_MR2,Demented,2,M,82,16,1.0,28.0,1693,0.694,1.037
369,OAS2_0185_MR3,Demented,3,M,86,16,1.0,26.0,1688,0.675,1.040
370,OAS2_0186_MR1,Nondemented,1,F,61,13,2.0,30.0,1319,0.801,1.331
371,OAS2_0186_MR2,Nondemented,2,F,63,13,2.0,30.0,1327,0.796,1.323


EDUC: Level of Education (min: 6, max: 23)

SES: Socioeconomic Status (1 to 5)

MMSE: Mini-Mental State Examination (min: 4, max:30)

eTIV: Estimated total intracranial volume (mm3)

nWBV: Normalized whole brain volume

ASF: Atlas scaling factor

In [32]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.compose import make_column_transformer
from sklearn import set_config; set_config(display='diagram')
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

preproc = ColumnTransformer([
    ('s_scaler', StandardScaler(), ['Age']),
    ('m_scaler', MinMaxScaler(), ['EDUC']),
    ('r_scaler', RobustScaler(), ['eTIV', 'MMSE']),
    ('lab_encoder', OneHotEncoder(), ['Group', 'M/F'])],
    remainder='passthrough')

preproc

ColumnTransformer(remainder='passthrough',
                  transformers=[('s_scaler', StandardScaler(), ['Age']),
                                ('m_scaler', MinMaxScaler(), ['EDUC']),
                                ('r_scaler', RobustScaler(), ['eTIV', 'MMSE']),
                                ('lab_encoder', OneHotEncoder(),
                                 ['Group', 'M/F'])])

In [52]:
oasis2_scaled=pd.DataFrame(preproc.fit_transform(X,y), columns = preproc.get_feature_names_out())
oasis2_scaled

,s_scaler__Age,m_scaler__EDUC,r_scaler__eTIV,r_scaler__MMSE,lab_encoder__Group_Converted,lab_encoder__Group_Demented,lab_encoder__Group_Nondemented,lab_encoder__M/F_F,lab_encoder__M/F_M,remainder__MRI ID,remainder__Visit,remainder__SES,remainder__nWBV,remainder__ASF
0,1.308738,0.470588,2.154167,-0.666667,0.0,0.0,1.0,0.0,1.0,OAS2_0001_MR1,1,2.0,0.696,0.883
1,1.439787,0.470588,2.225,0.333333,0.0,0.0,1.0,0.0,1.0,OAS2_0001_MR2,2,2.0,0.681,0.876
2,-0.263856,0.352941,0.866667,-2.0,0.0,1.0,0.0,0.0,1.0,OAS2_0002_MR1,1,2.0,0.736,1.046
3,-0.132806,0.352941,1.116667,-0.333333,0.0,1.0,0.0,0.0,1.0,OAS2_0002_MR2,2,2.0,0.713,1.01
4,0.391392,0.352941,0.95,-2.333333,0.0,1.0,0.0,0.0,1.0,OAS2_0002_MR3,3,2.0,0.701,1.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,0.653491,0.588235,0.929167,-0.333333,0.0,1.0,0.0,0.0,1.0,OAS2_0185_MR2,2,1.0,0.694,1.037
369,1.177688,0.588235,0.908333,-1.0,0.0,1.0,0.0,0.0,1.0,OAS2_0185_MR3,3,1.0,0.675,1.04
370,-2.098548,0.411765,-0.629167,0.333333,0.0,0.0,1.0,1.0,0.0,OAS2_0186_MR1,1,2.0,0.801,1.331
371,-1.836449,0.411765,-0.595833,0.333333,0.0,0.0,1.0,1.0,0.0,OAS2_0186_MR2,2,2.0,0.796,1.323


In [54]:
oasis2_scaled.rename(columns={"s_scaler__Age": "scaled_age",
                              "m_scaler__EDUC": "scaled_educ",
                              "r_scaler__eTIV": "scaled_eTIV",
                              "r_scaler__MMSE": "scaled_MMSE",
                              "lab_encoder__Group_Converted": "Converted_group",
                              "lab_encoder__Group_Demented": "demented-group",
                              "lab_encoder__Group_Nondemented": "non_demented_group",
                              "lab_encoder__M/F_F": "female",
                              "lab_encoder__M/F_M": "male",
                              "remainder__MRI ID": "MRI_id",
                              "remainder__Visit": "visit",
                              "remainder__SES": "SES",
                              "remainder__nWBV": "nWBV",
                              "remainder__ASF": "ASF"})

,scaled_age,scaled_educ,scaled_eTIV,scaled_MMSE,Converted_group,demented-group,non_demented_group,female,male,MRI_id,visit,SES,nWBV,ASF
0,1.308738,0.470588,2.154167,-0.666667,0.0,0.0,1.0,0.0,1.0,OAS2_0001_MR1,1,2.0,0.696,0.883
1,1.439787,0.470588,2.225,0.333333,0.0,0.0,1.0,0.0,1.0,OAS2_0001_MR2,2,2.0,0.681,0.876
2,-0.263856,0.352941,0.866667,-2.0,0.0,1.0,0.0,0.0,1.0,OAS2_0002_MR1,1,2.0,0.736,1.046
3,-0.132806,0.352941,1.116667,-0.333333,0.0,1.0,0.0,0.0,1.0,OAS2_0002_MR2,2,2.0,0.713,1.01
4,0.391392,0.352941,0.95,-2.333333,0.0,1.0,0.0,0.0,1.0,OAS2_0002_MR3,3,2.0,0.701,1.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,0.653491,0.588235,0.929167,-0.333333,0.0,1.0,0.0,0.0,1.0,OAS2_0185_MR2,2,1.0,0.694,1.037
369,1.177688,0.588235,0.908333,-1.0,0.0,1.0,0.0,0.0,1.0,OAS2_0185_MR3,3,1.0,0.675,1.04
370,-2.098548,0.411765,-0.629167,0.333333,0.0,0.0,1.0,1.0,0.0,OAS2_0186_MR1,1,2.0,0.801,1.331
371,-1.836449,0.411765,-0.595833,0.333333,0.0,0.0,1.0,1.0,0.0,OAS2_0186_MR2,2,2.0,0.796,1.323
